In [2]:
import praw
import pandas as pd
import requests
import time
import tqdm
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

True

In [4]:
api = praw.Reddit(client_id=os.getenv('client_id'),
                  client_secret=os.getenv('client_secret'), user_agent=os.getenv('user_agent'))


In [5]:
import pandas as pd
posts = []
sb = api.subreddit('roastme')
for post in sb.top(limit=10):
    posts.append([post.title, post.score, post.id, post.subreddit,
                 post.url, post.num_comments, post.selftext, post.created])
posts = pd.DataFrame(posts, columns=[
                     'title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
submission_ids = posts['id'].values

In [6]:
def get_comments(submission_id):
    submission = api.submission(id=submission_id)
    submission.comment_sort = 'top'
    submission.comment_limit = 1
    top_comment = submission.comments[0]
    return top_comment.body

In [7]:
# add comments to dataframe
comments = []
for submission_id in tqdm.tqdm(submission_ids):
    comments.append(get_comments(submission_id))
    time.sleep(1)
posts['comments'] = comments

100%|██████████| 10/10 [00:20<00:00,  2.01s/it]


In [8]:
posts.head()

,title,score,id,subreddit,url,num_comments,body,created,comments
0,Thank you for all the support! Seeing complete...,98092,amnyr9,RoastMe,https://i.redd.it/s2fi2wr7ibe21.jpg,2550,,1.549184e+09,Glad you're doing great OP. You're now the top...
1,Let's Roast the Coronavirus,77088,fnwcch,RoastMe,https://i.redd.it/1xgka63ywio41.png,3939,,1.585013e+09,Wish.com ebola.
2,Ten years ago I purchased a $50 Groupon for pr...,68636,ecw1ov,RoastMe,https://i.redd.it/qcrbkirlgm541.jpg,2255,,1.576775e+09,This is like the creepiest Brady Bunch intro I...
3,I roasted this baby picture my Mom printed on ...,58785,dwo2th,RoastMe,https://i.redd.it/kzrr5a1odty31.jpg,4480,,1.573808e+09,Well we now know at least one kid who wasn't m...
4,3.2 million karma and accepted to Harvard but ...,57513,bb6nwq,RoastMe,https://i.redd.it/d3ht20kk28r21.jpg,4981,,1.554809e+09,You look like the guy who does his mates’ math...


In [9]:
# get only title and comments 
df = posts[['title', 'comments']]
df.head()

,title,comments
0,Thank you for all the support! Seeing complete...,Glad you're doing great OP. You're now the top...
1,Let's Roast the Coronavirus,Wish.com ebola.
2,Ten years ago I purchased a $50 Groupon for pr...,This is like the creepiest Brady Bunch intro I...
3,I roasted this baby picture my Mom printed on ...,Well we now know at least one kid who wasn't m...
4,3.2 million karma and accepted to Harvard but ...,You look like the guy who does his mates’ math...


In [10]:
# save to csv
df.to_csv('../reddit_data.csv', index=False)